In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

# Importing pyod Packages

In [2]:
!pip install pyod
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


# Importing Metric Packages

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [4]:
mat_file_list = ['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [5]:
#Creating a Data Frame
df_columns = ['Data','#Samples','# Dimensions', 'Outlier Perc','ABOD','CBLOF','FB','HBOS','IForest','KNN','LOF','MCD','OCSVM','PCA']

In [6]:
#Creating a Empty Data Frame and Populating it with the above Column Names
#Region of Characteristics Data Frame / Performance Evaluation Table
roc_df = pd.DataFrame(columns=df_columns)
#Precision Data Frame / Performance Evaluation Table
prn_df = pd.DataFrame(columns=df_columns)
#Time Data Frame / Performance Evaluation Table
time_df = pd.DataFrame(columns=df_columns)


# Loading Mat File

In [7]:
data = loadmat('Anamoly_detec_data/cardio.mat')

In [8]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [9]:
len(data)

5

In [10]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [11]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

# Independent Feature Shape in Mat File

In [12]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

# Dependent Feature Shape in Mat File

In [13]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

In [14]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print('\n..... Processing', mat_file,'....')
    mat = loadmat(os.path.join('Anamoly_detec_data',mat_file))
    
    X = mat['X']
    y = mat['y'].ravel() #ravel() method to convert 1-D to 2-D
    outliers_fraction = np.count_nonzero(y)/len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    #Construct Containers for saving Results
    roc_list = [mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    prn_list = [mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    time_list = [mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    
    #Split the data for Training and Testing
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=random_state)
    
    #Standardizing Data for Processing
    X_train_norm, X_test_norm = standardizer(X_train,X_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)':ABOD(
    contamination=outliers_fraction),
                   'Cluster-based Local Outlier Factor': CBLOF(
                    contamination=outliers_fraction,check_estimator=False,
                   random_state=random_state),
                   'Feature Bagging':FeatureBagging(contamination=outliers_fraction,
                                                   random_state=random_state),
                   'Histogram-base Outlier Detection (HBOS)': HBOS(
                   contamination=outliers_fraction),
                   'Isolation Forest':IForest(contamination=outliers_fraction,random_state=random_state),
                   'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                   'Local Outlier Factor (LOF)':LOF(contamination=outliers_fraction),
                   'Minimum Covariance Determinant (MCD)':MCD(contamination=outliers_fraction,random_state=random_state),
                   'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                   'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction,random_state=random_state),
                }
    for clf_name,clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)
        roc = round(roc_auc_score(y_test,test_scores),ndigits=4)
        prn = round(precision_n_scores(y_test,test_scores),ndigits=4)
        
        print('{clf_name} ROC:{roc},precision @ rank n:{prn},execution time:{duration}s'.format(clf_name=clf_name,roc=roc,prn=prn,duration=duration))
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df,temp_df],axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df,temp_df],axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df,temp_df],axis=0)
    


..... Processing arrhythmia.mat ....
Angle-based Outlier Detector (ABOD) ROC:0.7687,precision @ rank n:0.3571,execution time:2.9173s
Cluster-based Local Outlier Factor ROC:0.7684,precision @ rank n:0.4643,execution time:3.285s
Feature Bagging ROC:0.7799,precision @ rank n:0.5,execution time:0.7383s
Histogram-base Outlier Detection (HBOS) ROC:0.8511,precision @ rank n:0.5714,execution time:2.5031s
Isolation Forest ROC:0.8527,precision @ rank n:0.5714,execution time:1.0487s
K Nearest Neighbors (KNN) ROC:0.782,precision @ rank n:0.5,execution time:0.1346s
Local Outlier Factor (LOF) ROC:0.7787,precision @ rank n:0.4643,execution time:0.1107s


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228,precision @ rank n:0.4286,execution time:6.4277s
One-class SVM (OCSVM) ROC:0.7986,precision @ rank n:0.5,execution time:0.1476s
Principal Component Analysis (PCA) ROC:0.7997,precision @ rank n:0.5,execution time:0.3368s

..... Processing cardio.mat ....
Angle-based Outlier Detector (ABOD) ROC:0.5763,precision @ rank n:0.1875,execution time:0.4987s
Cluster-based Local Outlier Factor ROC:0.8221,precision @ rank n:0.4844,execution time:0.2433s
Feature Bagging ROC:0.4879,precision @ rank n:0.1406,execution time:1.2081s
Histogram-base Outlier Detection (HBOS) ROC:0.8453,precision @ rank n:0.4688,execution time:0.01s
Isolation Forest ROC:0.9414,precision @ rank n:0.5,execution time:0.7401s
K Nearest Neighbors (KNN) ROC:0.6959,precision @ rank n:0.2812,execution time:0.2374s
Local Outlier Factor (LOF) ROC:0.4715,precision @ rank n:0.125,execution time:0.1431s


C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-152.076751858648862 > -250.756822706501453). You may want to try with a higher value of support_fraction (current value: 0.511).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC:0.8778,precision @ rank n:0.3906,execution time:1.0328s
One-class SVM (OCSVM) ROC:0.9507,precision @ rank n:0.5938,execution time:0.1247s
Principal Component Analysis (PCA) ROC:0.9638,precision @ rank n:0.6875,execution time:0.0798s

..... Processing glass.mat ....
Angle-based Outlier Detector (ABOD) ROC:0.7104,precision @ rank n:0.25,execution time:0.0818s
Cluster-based Local Outlier Factor ROC:0.8506,precision @ rank n:0.25,execution time:0.0768s
Feature Bagging ROC:0.7043,precision @ rank n:0.25,execution time:0.0648s
Histogram-base Outlier Detection (HBOS) ROC:0.6524,precision @ rank n:0.0,execution time:0.005s
Isolation Forest ROC:0.7195,precision @ rank n:0.25,execution time:0.5404s
K Nearest Neighbors (KNN) ROC:0.7805,precision @ rank n:0.25,execution time:0.0275s
Local Outlier Factor (LOF) ROC:0.7774,precision @ rank n:0.25,execution time:0.004s
Minimum Covariance Determinant (MCD) ROC:0.7165,precision @ rank n:0.0,execution time:0.1389s

...

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.3486,precision @ rank n:0.0,execution time:1.8275s
One-class SVM (OCSVM) ROC:0.4972,precision @ rank n:0.0,execution time:2.1377s
Principal Component Analysis (PCA) ROC:0.504,precision @ rank n:0.0,execution time:0.0708s

..... Processing pendigits.mat ....
Angle-based Outlier Detector (ABOD) ROC:0.7008,precision @ rank n:0.0308,execution time:2.4532s
Cluster-based Local Outlier Factor ROC:0.9609,precision @ rank n:0.3077,execution time:0.4259s
Feature Bagging ROC:0.4687,precision @ rank n:0.0462,execution time:7.0616s
Histogram-base Outlier Detection (HBOS) ROC:0.9294,precision @ rank n:0.2615,execution time:0.014s
Isolation Forest ROC:0.9422,precision @ rank n:0.2769,execution time:1.081s
K Nearest Neighbors (KNN) ROC:0.7602,precision @ rank n:0.0462,execution time:0.971s
Local Outlier Factor (LOF) ROC:0.481,precision @ rank n:0.0462,execution time:0.8585s
Minimum Covariance Determinant (MCD) ROC:0.8271,precision @ rank n:0.0615,execution ti

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.541490230701029 > -75.592253998160075). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.022992269210960 > -76.150022952275933). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.588090860494539 > -73.997089087121466). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
C

C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.810409699567977 > -116.332661175263638). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.246740593074335 > -75.373826826202034). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\LENOVO\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.256978425484334 > -76.476983289741668). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)


Minimum Covariance Determinant (MCD) ROC:0.9903,precision @ rank n:0.7534,execution time:17.2539s
One-class SVM (OCSVM) ROC:0.9922,precision @ rank n:0.9553,execution time:73.5094s
Principal Component Analysis (PCA) ROC:0.9902,precision @ rank n:0.9503,execution time:0.0559s

..... Processing vertebral.mat ....
Angle-based Outlier Detector (ABOD) ROC:0.2797,precision @ rank n:0.0,execution time:0.0559s
Cluster-based Local Outlier Factor ROC:0.3908,precision @ rank n:0.0,execution time:0.0698s
Feature Bagging ROC:0.3027,precision @ rank n:0.0,execution time:0.0638s
Histogram-base Outlier Detection (HBOS) ROC:0.2695,precision @ rank n:0.0,execution time:0.003s
Isolation Forest ROC:0.3576,precision @ rank n:0.0,execution time:0.4249s
K Nearest Neighbors (KNN) ROC:0.318,precision @ rank n:0.0,execution time:0.013s
Local Outlier Factor (LOF) ROC:0.318,precision @ rank n:0.0,execution time:0.006s
Minimum Covariance Determinant (MCD) ROC:0.3308,precision @ rank n:0.0,execution time:0.0638s
On

In [15]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [16]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385


In [17]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,2.9173,3.285,0.7383,2.5031,1.0487,0.1346,0.1107,6.4277,0.1476,0.3368
0,cardio,1831,21,9.6122,0.4987,0.2433,1.2081,0.01,0.7401,0.2374,0.1431,1.0328,0.1247,0.0798
0,glass,214,9,4.2056,0.0818,0.0768,0.0648,0.005,0.5404,0.0275,0.004,0.1389,0.003,0.013
0,ionosphere,351,33,35.8974,0.1406,0.0848,0.1077,0.017,0.5305,0.0339,0.014,0.1277,0.007,0.04
0,letter,1600,32,6.25,0.583,0.2154,1.1544,0.0175,0.7201,0.2523,0.1506,1.8626,0.1207,0.006
0,lympho,148,18,4.0541,0.0474,0.0748,0.0529,0.01,0.5205,0.017,0.005,0.0808,0.003,0.007
0,mnist,7603,100,9.2069,11.3216,1.6074,81.3263,0.0748,2.8135,9.8567,9.7058,4.0907,7.109,0.2224
0,musk,3062,166,3.1679,3.7678,0.5411,20.5973,0.1002,1.9638,3.2328,2.8058,16.0607,1.7761,0.2025
0,optdigits,5216,64,2.8758,4.2268,0.7071,22.2673,0.0478,1.3746,2.7968,2.4684,1.8275,2.1377,0.0708
0,pendigits,6870,16,2.2707,2.4532,0.4259,7.0616,0.014,1.081,0.971,0.8585,3.181,1.3725,0.016
